# Day8_5: 다변량 시계열 분석 (VAR + 그랜저 인과성)

---

## 📚 학습 목표

**Part 1: 기초 - 그랜저 인과성 검정**
1. 상관관계 vs 인과관계 차이 이해하기
2. 그랜저 인과성(Granger Causality) 개념 파악하기
3. statsmodels로 그랜저 인과성 검정 수행하기
4. 양방향 인과성 검정 및 해석하기
5. 경제 데이터로 인과관계 분석 실습하기

**Part 2: 심화 - VAR 모델**
1. VAR(Vector AutoRegression) 모델 이해하기
2. 최적 시차(lag) 결정하기 (AIC, BIC)
3. VAR 모델 학습 및 다변량 예측하기
4. 차분 복원(역변환)으로 원본 스케일 예측하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| **그랜저 인과성** | 변수 간 인과관계 규명 | 광고비 → 매출? 금리 → 주가? |
| **VAR 모델** | 여러 변수 동시 예측 | 경제지표, 다중 종목 주가 |
| **상호작용 분석** | 변수 간 영향력 파악 | A 부서 성과 → B 부서 성과? |
| **정책 효과 분석** | 정책의 파급 효과 | 금리 인상 → 환율/물가 영향 |

**분석가 관점**: 단변량 ARIMA는 한 변수만 예측하지만, 현실에서는 여러 변수가 서로 영향을 주고받습니다. VAR과 그랜저 검정은 이러한 복잡한 관계를 분석하는 핵심 도구입니다.

---

## 🔧 환경 설정

In [ ]:
# 필요한 라이브러리 설치 (필요시 주석 해제)
# !pip install statsmodels yfinance plotly pandas numpy

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 시계열 분석 라이브러리
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller, grangercausalitytests

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

---

# Part 1: 기초 - 그랜저 인과성 검정

---

## 1.1 상관관계 vs 인과관계

### 중요한 구분

| 개념 | 의미 | 예시 |
|------|------|------|
| **상관관계** | 두 변수가 함께 움직임 | 아이스크림 판매 ↔ 익사 사고 |
| **인과관계** | 한 변수가 다른 변수에 영향 | 광고비 → 매출 |

### 상관관계 ≠ 인과관계

"아이스크림 판매량과 익사 사고는 상관관계가 있다"
- 둘 다 **여름(숨은 변수)**에 증가
- 아이스크림이 익사를 유발하는 것이 아님!

### 그랜저 인과성이란?

**"X가 Y를 예측하는 데 도움이 되는가?"**

- 엄밀한 인과관계는 아님 ("예측 인과성")
- 시간적 선행관계 기반
- "X의 과거 값이 Y 예측에 유의미한 정보를 제공하는가?"

### 💡 실무 비유

**마케팅 분석 시나리오**

| 질문 | 분석 방법 | 결과 해석 |
|------|----------|----------|
| "광고비와 매출이 연관되어 있나?" | 상관분석 | r=0.8 (강한 양의 상관) |
| "광고비가 매출을 일으키는가?" | 그랜저 검정 | 광고비 → 매출 (예측력 있음) |
| "매출이 광고비에 영향을 주나?" | 역방향 검정 | 매출 → 광고비? (예산 배정) |

---

## 1.2 실습 데이터 준비

M2 통화량(Money)과 개인소비지출(Spending) 데이터를 사용합니다.

In [ ]:
# 데이터 생성 (Money Stock & Spending 시뮬레이션)
# 실제로는 FRED API 등에서 가져올 수 있습니다
np.random.seed(42)

# 날짜 인덱스 생성
dates = pd.date_range(start='2010-01-01', periods=150, freq='MS')

# Money: 추세 + 랜덤
money_trend = np.linspace(1000, 2500, 150)
money = money_trend + np.cumsum(np.random.randn(150) * 30)

# Spending: Money에 지연 반응 + 자체 트렌드
spending_trend = np.linspace(800, 1800, 150)
spending = spending_trend + 0.3 * np.roll(money, 3) + np.cumsum(np.random.randn(150) * 20)

# DataFrame 생성
df_econ = pd.DataFrame({
    'Money': money,
    'Spending': spending
}, index=dates)

print(f"데이터 기간: {df_econ.index[0].date()} ~ {df_econ.index[-1].date()}")
print(f"데이터 개수: {len(df_econ)}")
df_econ.head()

In [ ]:
# 시계열 시각화
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=['M2 통화량 (Money)', '개인소비지출 (Spending)'])

fig.add_trace(go.Scatter(x=df_econ.index, y=df_econ['Money'],
                         mode='lines', name='Money', line=dict(color='blue')),
              row=1, col=1)

fig.add_trace(go.Scatter(x=df_econ.index, y=df_econ['Spending'],
                         mode='lines', name='Spending', line=dict(color='green')),
              row=2, col=1)

fig.update_layout(title='경제 지표 시계열', height=500)
fig.show()

In [ ]:
# 상관관계 확인
correlation = df_econ['Money'].corr(df_econ['Spending'])
print(f"Money와 Spending의 상관계수: {correlation:.4f}")
print("\n=> 강한 양의 상관관계! 하지만 인과관계는 별도로 검정 필요")

---

## 1.3 그랜저 인과성 검정

### 가설 설정

- **귀무가설 (H0)**: X가 Y를 그랜저-원인하지 않는다 (X의 과거값이 Y 예측에 도움이 안 됨)
- **대립가설 (H1)**: X가 Y를 그랜저-원인한다 (X의 과거값이 Y 예측에 도움이 됨)

### p-value 해석

- **p < 0.05**: 귀무가설 기각 → X가 Y를 그랜저-원인함
- **p >= 0.05**: 귀무가설 채택 → 인과관계 없음

In [ ]:
# 정상성 확보를 위한 차분
def adf_test(series, name=''):
    """ADF 검정"""
    result = adfuller(series.dropna())
    print(f'{name}: ADF={result[0]:.4f}, p-value={result[1]:.4f}', end='')
    print(" -> 정상" if result[1] < 0.05 else " -> 비정상")
    return result[1]

print("=== 원본 데이터 정상성 ===")
adf_test(df_econ['Money'], 'Money')
adf_test(df_econ['Spending'], 'Spending')

# 1차 차분
df_diff = df_econ.diff().dropna()

print("\n=== 1차 차분 후 정상성 ===")
adf_test(df_diff['Money'], 'Money_diff')
adf_test(df_diff['Spending'], 'Spending_diff')

In [ ]:
def granger_causality_test(data, x_col, y_col, maxlag=12):
    """
    그랜저 인과성 검정 수행
    
    Parameters:
    -----------
    data : pd.DataFrame
        시계열 데이터 (정상성 확보 필요)
    x_col : str
        원인 변수 (X → Y 검정에서 X)
    y_col : str
        결과 변수
    maxlag : int
        검정할 최대 시차
    
    Returns:
    --------
    dict : lag별 p-value
    """
    print(f"\n=== 그랜저 인과성 검정: {x_col} → {y_col} ===")
    
    # grangercausalitytests는 [Y, X] 순서로 입력
    # Y를 예측할 때 X가 도움이 되는지 검정
    test_data = data[[y_col, x_col]].dropna()
    
    # 검정 수행 (verbose=False로 상세 출력 숨김)
    results = grangercausalitytests(test_data, maxlag=maxlag, verbose=False)
    
    # 결과 정리
    p_values = {}
    print(f"\n{'Lag':<5} {'F-statistic':<15} {'p-value':<12} {'유의성':<10}")
    print("-" * 45)
    
    for lag in range(1, maxlag + 1):
        # ssr_ftest 결과 사용 (F-검정)
        f_stat = results[lag][0]['ssr_ftest'][0]
        p_val = results[lag][0]['ssr_ftest'][1]
        p_values[lag] = p_val
        
        significance = "***" if p_val < 0.01 else "**" if p_val < 0.05 else "*" if p_val < 0.1 else ""
        print(f"{lag:<5} {f_stat:<15.4f} {p_val:<12.4f} {significance}")
    
    # 가장 유의한 시차 찾기
    best_lag = min(p_values, key=p_values.get)
    best_p = p_values[best_lag]
    
    print("\n결론:")
    if best_p < 0.05:
        print(f"  => '{x_col}'가 '{y_col}'를 그랜저-원인합니다 (lag={best_lag}, p={best_p:.4f})")
    else:
        print(f"  => '{x_col}'가 '{y_col}'를 그랜저-원인하지 않습니다 (p >= 0.05)")
    
    return p_values

# Money → Spending 검정
p_vals_1 = granger_causality_test(df_diff, 'Money', 'Spending', maxlag=8)

In [ ]:
# 역방향 검정: Spending → Money
p_vals_2 = granger_causality_test(df_diff, 'Spending', 'Money', maxlag=8)

### 결과 해석

| 검정 방향 | 결과 | 해석 |
|----------|------|------|
| Money → Spending | p < 0.05 | 통화량이 소비지출에 선행 |
| Spending → Money | p >= 0.05 | 소비지출은 통화량 예측에 도움 안 됨 |

**경제학적 해석**: 
- 통화량 증가 → (시차를 두고) → 소비지출 증가
- 통화정책이 실물경제에 영향을 미침

---

## 1.4 주식 데이터로 인과성 분석

실제 주식 데이터에서 인과관계를 분석해봅니다.

In [ ]:
import yfinance as yf

# 관련 종목 데이터 수집: 삼성전자 vs SK하이닉스
samsung = yf.download('005930.KS', start='2022-01-01', end='2024-12-31')['Close']
skhynix = yf.download('000660.KS', start='2022-01-01', end='2024-12-31')['Close']

# DataFrame 결합
df_stock = pd.DataFrame({
    'Samsung': samsung,
    'SKHynix': skhynix
}).dropna()

print(f"데이터 기간: {df_stock.index[0].date()} ~ {df_stock.index[-1].date()}")
print(f"데이터 개수: {len(df_stock)}")

In [ ]:
# 시각화
fig = make_subplots(rows=1, cols=1)

# 정규화 (시작점 = 100)
samsung_norm = df_stock['Samsung'] / df_stock['Samsung'].iloc[0] * 100
skhynix_norm = df_stock['SKHynix'] / df_stock['SKHynix'].iloc[0] * 100

fig.add_trace(go.Scatter(x=df_stock.index, y=samsung_norm,
                         name='삼성전자', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df_stock.index, y=skhynix_norm,
                         name='SK하이닉스', line=dict(color='red')))

fig.update_layout(title='삼성전자 vs SK하이닉스 주가 추이 (정규화)',
                  yaxis_title='상대 가격 (시작=100)')
fig.show()

In [ ]:
# 수익률로 변환 (정상성 확보 + 스케일 통일)
df_returns = df_stock.pct_change().dropna() * 100  # 백분율 수익률

# 정상성 확인
print("=== 수익률 데이터 정상성 ===")
adf_test(df_returns['Samsung'], 'Samsung 수익률')
adf_test(df_returns['SKHynix'], 'SKHynix 수익률')

In [ ]:
# 양방향 그랜저 인과성 검정
print("\n" + "="*60)
print("반도체 주식 간 그랜저 인과성 분석")
print("="*60)

# 삼성전자 → SK하이닉스
granger_causality_test(df_returns, 'Samsung', 'SKHynix', maxlag=5)

# SK하이닉스 → 삼성전자
granger_causality_test(df_returns, 'SKHynix', 'Samsung', maxlag=5)

### 💡 주식 인과성 해석 주의사항

1. **리드-래그 관계**: 큰 종목이 작은 종목에 선행하는 경향
2. **효율적 시장**: 이론적으로 주가는 즉시 정보 반영 (인과성 약함)
3. **공통 요인**: 반도체 업황, 환율 등 제3의 변수 존재
4. **투자 주의**: 그랜저 인과성 ≠ 수익 보장

---

# Part 2: 심화 - VAR 모델

---

## 2.1 VAR 모델 이해

### VAR(Vector AutoRegression)이란?

**여러 시계열 변수**가 서로 영향을 주고받는 관계를 모델링하는 **다변량** 시계열 분석 기법입니다.

### 단변량 vs 다변량

| 특성 | ARIMA (단변량) | VAR (다변량) |
|------|---------------|-------------|
| 변수 수 | 1개 | 2개 이상 |
| 예측 대상 | Y의 미래값 | Y1, Y2, ... 모두 |
| 상호작용 | 없음 | 변수 간 영향 반영 |
| 활용 | 단일 KPI 예측 | 연관 지표 동시 예측 |

### VAR(p) 모델 수식

2개 변수 (Y1, Y2)인 경우:

$$Y_{1,t} = c_1 + \phi_{11,1}Y_{1,t-1} + \phi_{12,1}Y_{2,t-1} + \cdots + \epsilon_{1,t}$$
$$Y_{2,t} = c_2 + \phi_{21,1}Y_{1,t-1} + \phi_{22,1}Y_{2,t-1} + \cdots + \epsilon_{2,t}$$

→ 각 변수가 **자신의 과거**뿐만 아니라 **다른 변수의 과거**에도 영향받음

### 💡 실무 비유

**부서 간 성과 분석**

| 변수 | 의미 |
|------|------|
| Y1 = 마케팅 성과 | 이번 달 마케팅 성과 |
| Y2 = 영업 성과 | 이번 달 영업 성과 |

**VAR 모델 해석:**
- 마케팅 성과 = 지난달 마케팅 + 지난달 영업 + 오차
- 영업 성과 = 지난달 영업 + 지난달 마케팅 + 오차

→ 두 부서가 서로 영향을 주고받는 관계 모델링!

---

## 2.2 VAR 모델 구축

In [ ]:
# 경제 데이터 사용 (앞서 생성한 df_econ)
# 정상성을 위해 1차 차분 적용
df_diff = df_econ.diff().dropna()

# 훈련/테스트 분할
n_test = 12  # 테스트 기간
train_var = df_diff[:-n_test]
test_var = df_diff[-n_test:]

print(f"훈련 데이터: {len(train_var)}개")
print(f"테스트 데이터: {len(test_var)}개")

In [ ]:
# VAR 모델 생성 및 최적 시차 선택
model_var = VAR(train_var)

# 최적 시차(lag) 결정 - AIC, BIC 등 기준
lag_selection = model_var.select_order(maxlags=15)
print("=== 최적 시차 선택 ===")
print(lag_selection.summary())

### 시차 선택 기준 해석

| 기준 | 특징 |
|------|------|
| **AIC** | 복잡도 페널티 약함, 과적합 가능 |
| **BIC** | 복잡도 페널티 강함, 단순 모델 선호 |
| **HQIC** | AIC와 BIC 중간 |
| **FPE** | 예측 오차 최소화 |

일반적으로 **AIC 또는 BIC** 기준의 최적 시차 사용

In [ ]:
# AIC 기준 최적 시차로 모델 학습
optimal_lag = lag_selection.aic
print(f"AIC 기준 최적 시차: {optimal_lag}")

# VAR 모델 학습
fitted_var = model_var.fit(optimal_lag)
print("\n모델 학습 완료!")
print(fitted_var.summary())

---

## 2.3 VAR 예측 및 역변환

In [ ]:
# 예측 수행
# VAR 예측에는 마지막 lag 개의 데이터 필요
lag_order = fitted_var.k_ar
forecast_input = train_var.values[-lag_order:]

# n_test 기간 예측
forecast_var = fitted_var.forecast(y=forecast_input, steps=n_test)

# 예측 결과를 DataFrame으로
df_forecast = pd.DataFrame(
    forecast_var,
    index=test_var.index,
    columns=[col + '_diff_pred' for col in train_var.columns]
)

print("=== 차분 데이터 예측 결과 ===")
df_forecast.head()

In [ ]:
def invert_diff_forecast(original_df, forecast_diff, n_diff=1):
    """
    차분된 예측값을 원본 스케일로 역변환
    
    Parameters:
    -----------
    original_df : pd.DataFrame
        원본 데이터 (차분 전)
    forecast_diff : np.array or pd.DataFrame
        차분된 예측값
    n_diff : int
        차분 횟수
    
    Returns:
    --------
    pd.DataFrame : 원본 스케일 예측값
    """
    if isinstance(forecast_diff, np.ndarray):
        forecast_diff = pd.DataFrame(
            forecast_diff,
            columns=original_df.columns
        )
    
    # 마지막 원본값
    last_values = original_df.iloc[-1]
    
    # 누적합으로 역변환
    # 1차 차분: Y(t) = Y(t-1) + diff(t)
    forecast_original = forecast_diff.cumsum() + last_values.values
    forecast_original.columns = [col + '_forecast' for col in original_df.columns]
    
    return forecast_original

# 원본 스케일로 역변환
# 훈련 기간의 마지막 값 기준 (테스트 시작 직전)
train_original = df_econ.iloc[:-n_test]

df_forecast_original = invert_diff_forecast(
    train_original,
    forecast_var
)
df_forecast_original.index = test_var.index

print("=== 원본 스케일 예측 결과 ===")
df_forecast_original.head()

In [ ]:
# 예측 결과 시각화
test_original = df_econ.iloc[-n_test:]

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=['Money (통화량)', 'Spending (소비지출)'])

# Money
fig.add_trace(go.Scatter(
    x=df_econ.index, y=df_econ['Money'],
    mode='lines', name='Money 실제',
    line=dict(color='blue')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_forecast_original.index, y=df_forecast_original['Money_forecast'],
    mode='lines', name='Money 예측',
    line=dict(color='blue', dash='dash')
), row=1, col=1)

# Spending
fig.add_trace(go.Scatter(
    x=df_econ.index, y=df_econ['Spending'],
    mode='lines', name='Spending 실제',
    line=dict(color='green')
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=df_forecast_original.index, y=df_forecast_original['Spending_forecast'],
    mode='lines', name='Spending 예측',
    line=dict(color='green', dash='dash')
), row=2, col=1)

fig.update_layout(title='VAR 모델 예측 결과', height=600)
fig.show()

In [ ]:
# 예측 성능 평가
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_var_forecast(actual, predicted):
    """VAR 예측 성능 평가"""
    results = {}
    for col in actual.columns:
        pred_col = col + '_forecast'
        if pred_col in predicted.columns:
            mae = mean_absolute_error(actual[col], predicted[pred_col])
            rmse = np.sqrt(mean_squared_error(actual[col], predicted[pred_col]))
            mape = np.mean(np.abs((actual[col].values - predicted[pred_col].values) / actual[col].values)) * 100
            results[col] = {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}
    return results

metrics = evaluate_var_forecast(test_original, df_forecast_original)

print("=== VAR 예측 성능 ===")
for var, m in metrics.items():
    print(f"\n{var}:")
    print(f"  MAE: {m['MAE']:.2f}")
    print(f"  RMSE: {m['RMSE']:.2f}")
    print(f"  MAPE: {m['MAPE']:.2f}%")

---

## 2.4 VAR 모델 진단

In [ ]:
# 잔차 자기상관 검정 (Durbin-Watson)
from statsmodels.stats.stattools import durbin_watson

residuals = fitted_var.resid
print("=== Durbin-Watson 검정 ===")
print("(2에 가까울수록 자기상관 없음)\n")

for i, col in enumerate(train_var.columns):
    dw = durbin_watson(residuals[:, i])
    print(f"{col}: DW = {dw:.4f}")

In [ ]:
# 계수 해석
print("=== VAR 모델 계수 ===")
print("\n각 변수에 대한 회귀 계수:")

for i, target in enumerate(train_var.columns):
    print(f"\n[{target} 방정식]")
    coef_df = pd.DataFrame(
        fitted_var.coefs.reshape(-1, len(train_var.columns)).T,
        index=train_var.columns
    )
    print(coef_df)

---

# 🎯 실습 퀴즈

---

## 퀴즈 1 ⭐

상관관계와 인과관계의 차이점을 설명하고, 그랜저 인과성 검정의 귀무가설을 작성하세요.

```python
# 빈칸을 채우세요
# 상관관계: 두 변수가 함께 ______는 것
# 인과관계: 한 변수가 다른 변수에 ______을 주는 것

# 그랜저 인과성 귀무가설 (H0):
# "X가 Y를 그랜저-______하지 않는다"
# = "X의 과거값이 Y 예측에 ______이 안 된다"
```

In [ ]:
# 퀴즈 1 풀이


## 퀴즈 2 ⭐

그랜저 인과성 검정 결과 p-value가 0.03이 나왔습니다. 이 결과를 어떻게 해석해야 하나요?

```python
# p-value = 0.03 해석
p_value = 0.03
alpha = 0.05  # 유의수준

if p_value ______ alpha:
    result = "귀무가설 ______"
    interpretation = "X가 Y를 그랜저-______한다"
else:
    result = "귀무가설 ______"
    interpretation = "인과관계 ______"

print(f"결과: {result}")
print(f"해석: {interpretation}")
```

In [ ]:
# 퀴즈 2 풀이


## 퀴즈 3 ⭐⭐

statsmodels의 grangercausalitytests 함수를 사용하여 그랜저 인과성 검정을 수행하세요.

주의: 함수 입력 순서는 [Y, X]입니다 (Y를 예측할 때 X가 도움이 되는지 검정)

```python
from statsmodels.tsa.stattools import grangercausalitytests

# 데이터 준비 (df_diff에 'Money', 'Spending' 열이 있다고 가정)
# "Money가 Spending을 그랜저-원인하는가?" 검정

# grangercausalitytests 입력: [결과변수, 원인변수]
test_data = df_diff[[______, ______]]  # 순서 주의!

# 검정 수행 (maxlag=5)
results = grangercausalitytests(test_data, maxlag=______, verbose=True)
```

In [ ]:
# 퀴즈 3 풀이


## 퀴즈 4 ⭐⭐

VAR 모델과 ARIMA 모델의 차이점을 코드로 비교하세요.

```python
# ARIMA: 단변량 (1개 변수)
from statsmodels.tsa.arima.model import ARIMA
arima_model = ARIMA(df['______'], order=(1,1,1))  # 단일 열

# VAR: 다변량 (2개 이상 변수)
from statsmodels.tsa.api import VAR
var_model = VAR(df[['______', '______']])  # 여러 열

# VAR의 장점은?
# 변수 간 ______을 반영하여 예측
```

In [ ]:
# 퀴즈 4 풀이


## 퀴즈 5 ⭐⭐

VAR 모델의 최적 시차(lag)를 선택하는 코드를 작성하세요.

```python
from statsmodels.tsa.api import VAR

# VAR 모델 생성
model = VAR(df_diff)

# 최적 시차 선택
lag_selection = model._______(maxlags=15)

# AIC 기준 최적 시차
optimal_lag = lag_selection._____
print(f"AIC 기준 최적 시차: {optimal_lag}")

# BIC 기준 최적 시차
optimal_lag_bic = lag_selection._____
print(f"BIC 기준 최적 시차: {optimal_lag_bic}")
```

In [ ]:
# 퀴즈 5 풀이


## 퀴즈 6 ⭐⭐⭐

VAR 모델을 학습하고 예측을 수행하는 전체 코드를 작성하세요.

```python
from statsmodels.tsa.api import VAR

# 1. 데이터 분할
n_test = 12
train = df_diff[:-n_test]
test = df_diff[-n_test:]

# 2. VAR 모델 학습
model = VAR(______)
fitted = model.fit(______)

# 3. 예측 (forecast 메서드 사용)
lag_order = fitted.k_ar
forecast_input = train.values[-______:]
forecast = fitted.forecast(y=forecast_input, steps=______)

print(f"예측 shape: {forecast.shape}")
```

In [ ]:
# 퀴즈 6 풀이


## 퀴즈 7 ⭐⭐⭐

yfinance를 사용하여 2개 종목의 주가를 수집하고, 양방향 그랜저 인과성 검정을 수행하세요.

요구사항:
1. 네이버(NAVER)와 카카오(KAKAO) 주식 수집
2. 수익률로 변환 (정상성 확보)
3. 양방향 검정: NAVER → KAKAO, KAKAO → NAVER

```python
import yfinance as yf

# 1. 데이터 수집 (한국 주식)
naver = yf.download('035420.KS', period='2y')['Close']
kakao = yf.download('035720.KS', period='2y')['Close']

# 2. DataFrame 결합 및 수익률 변환
# 여기에 코드 작성

# 3. 양방향 그랜저 검정
# 여기에 코드 작성
```

In [ ]:
# 퀴즈 7 풀이


## 퀴즈 8 ⭐⭐⭐⭐

차분된 VAR 예측 결과를 원본 스케일로 역변환하는 함수를 구현하세요.

힌트: 1차 차분의 역변환은 누적합 + 마지막 원본값

```python
def invert_difference(original_series, forecast_diff):
    """
    차분된 예측값을 원본 스케일로 역변환
    
    Parameters:
    -----------
    original_series : pd.Series
        원본 시계열 (차분 전)
    forecast_diff : np.array
        차분된 예측값
    
    Returns:
    --------
    np.array : 원본 스케일 예측값
    """
    # 마지막 원본값
    last_value = original_series.iloc[______]
    
    # 누적합으로 역변환
    # diff(t) = Y(t) - Y(t-1) 이므로
    # Y(t) = Y(t-1) + diff(t)
    forecast_original = np._______(forecast_diff) + last_value
    
    return forecast_original

# 테스트
# original = pd.Series([100, 110, 125, 140])
# diff_forecast = np.array([15, 12, 18])  # 예측된 차분
# result = invert_difference(original, diff_forecast)
# 예상 결과: [155, 167, 185] (140+15=155, 155+12=167, ...)
```

In [ ]:
# 퀴즈 8 풀이


## 퀴즈 9 ⭐⭐⭐⭐

여러 변수에 대한 그랜저 인과성 매트릭스를 생성하는 함수를 구현하세요.

```python
def granger_matrix(data, variables, maxlag=5):
    """
    여러 변수 간 그랜저 인과성 매트릭스 생성
    
    Parameters:
    -----------
    data : pd.DataFrame
        시계열 데이터
    variables : list
        분석할 변수 목록
    maxlag : int
        검정할 최대 시차
    
    Returns:
    --------
    pd.DataFrame : 인과성 매트릭스 (행: 원인, 열: 결과)
    """
    n = len(variables)
    matrix = np.zeros((n, n))
    
    for i, cause in enumerate(variables):
        for j, effect in enumerate(variables):
            if i != j:  # 자기 자신 제외
                # 그랜저 검정 수행
                # 여기에 코드 작성
                pass
    
    return pd.DataFrame(matrix, index=variables, columns=variables)

# 사용 예시
# matrix = granger_matrix(df_returns, ['Samsung', 'SKHynix', 'Naver'])
# 시각화: heatmap으로 표시
```

In [ ]:
# 퀴즈 9 풀이


## 퀴즈 10 ⭐⭐⭐⭐⭐

**종합 프로젝트**: 다변량 시계열 분석 파이프라인 클래스를 구현하세요.

요구사항:
1. 클래스명: `MultivariateAnalyzer`
2. 메서드:
   - `fit(data, test_size)`: 데이터 분할 및 모델 학습
   - `granger_test(x, y)`: 그랜저 인과성 검정
   - `predict(steps)`: VAR 예측
   - `evaluate()`: 성능 평가
   - `plot()`: 예측 결과 시각화
3. 자동 정상성 변환 (차분) 지원
4. 역변환으로 원본 스케일 예측

```python
class MultivariateAnalyzer:
    def __init__(self, maxlag=15):
        self.maxlag = maxlag
        # 추가 속성들...
    
    def fit(self, data, test_size=12):
        """데이터 분할, 정상성 변환, VAR 학습"""
        pass
    
    def granger_test(self, cause, effect, maxlag=None):
        """그랜저 인과성 검정"""
        pass
    
    def predict(self, steps=None):
        """VAR 예측 (원본 스케일 반환)"""
        pass
    
    def evaluate(self):
        """예측 성능 평가"""
        pass
    
    def plot(self):
        """시각화"""
        pass

# 사용 예시
analyzer = MultivariateAnalyzer()
analyzer.fit(df_econ, test_size=12)
analyzer.granger_test('Money', 'Spending')
analyzer.predict()
metrics = analyzer.evaluate()
analyzer.plot()
```

In [ ]:
# 퀴즈 10 풀이


---

## 📊 학습 정리

---

### Part 1: 기초 핵심 요약

| 개념 | 설명 | 코드 예시 |
|------|------|----------|
| **상관 vs 인과** | 함께 움직임 vs 영향 관계 | correlation ≠ causation |
| **그랜저 인과성** | X의 과거가 Y 예측에 도움? | `grangercausalitytests()` |
| **귀무가설** | X가 Y를 원인하지 않음 | p < 0.05면 기각 |
| **양방향 검정** | X→Y, Y→X 모두 확인 | 피드백 관계 파악 |
| **정상성 필수** | 차분 후 검정 | `adfuller()` + `diff()` |

### Part 2: 심화 핵심 요약

| 개념 | 설명 | 코드 예시 |
|------|------|----------|
| **VAR** | 다변량 자기회귀 | `VAR(df).fit(lag)` |
| **최적 시차** | AIC/BIC 최소화 | `select_order(maxlags=15)` |
| **예측** | 여러 변수 동시 예측 | `fitted.forecast(steps=n)` |
| **역변환** | 차분 → 원본 스케일 | `cumsum() + last_value` |

### 💡 실무 팁

1. **정상성 먼저**: VAR도 정상 시계열 필요 (차분 적용)
2. **그랜저 ≠ 진짜 인과**: "예측 인과성"이며 숨은 변수 가능
3. **양방향 검정**: 한쪽만 보지 말고 역방향도 확인
4. **시차 선택**: AIC와 BIC 모두 참고, 해석 가능한 시차 선호
5. **역변환 주의**: 다중 차분 시 순서대로 역변환
6. **변수 개수**: 너무 많으면 과적합, 2-5개 권장

---

## Day8 시계열 분석 시리즈 완료

| Day | 주제 | 핵심 내용 |
|-----|------|----------|
| **Day8_0** | 시계열 기초 | datetime, DatetimeIndex, shift, diff |
| **Day8_1** | 분석 기법 | resample, rolling, 계절분해 |
| **Day8_2** | 시각화+프로젝트 | Plotly, yfinance, 대시보드 |
| **Day8_3** | 정상성 진단 | ADF, KPSS, ACF/PACF |
| **Day8_4** | ARIMA/SARIMA | 예측 모델링, auto_arima |
| **Day8_5** | VAR+그랜저 | 다변량 분석, 인과성 검정 |